### 1. Import dependencies

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

import torchfunc
import flowDatasetV4
from torch.utils.data.sampler import SubsetRandomSampler

import wandb


In [2]:
torch.__version__
torchfunc.cuda.reset()
torch.cuda.empty_cache()

### 2. Create PyTorch data generators

In [3]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        #transforms.Resize((500,500)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        #transforms.Resize((500,500)),
        transforms.ToTensor(),
        normalize
    ]),
}


In [4]:
train_dataset = flowDatasetV4.flowDataset('x',False,0.2,data_transforms['train'])
test_dataset = flowDatasetV4.flowDataset('x',True,0.2,data_transforms['validation'])

In [5]:
batch_size = 5

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle =True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,  shuffle=True)

In [6]:
dataloaders = {
    'train': train_loader,
    'validation': test_loader
}

### 3. Create the network

In [7]:
device = torch.device("cuda:0")

In [8]:
model = models.resnet101(pretrained=True).to(device)
    
for param in model.parameters():
    param.requires_grad = False   
    
model.fc = nn.Sequential(
               nn.Linear(2048, 2048),
               nn.ReLU(inplace=True),
               nn.Linear(2048, 1024),
               nn.ReLU(inplace=True),
               nn.Linear(1024, 128),
               nn.Identity(inplace=True),
               nn.Linear(128, 1)).to(device)


# save_path = 'models/pytorch/weights_run_31.h5'
# model.load_state_dict(torch.load(save_path))

### 4. Train the model

In [ ]:
#Log Hyperparameters
num_epochs = 100
lr = 0.000001
batch_size = batch_size


wandb.config.epochs = num_epochs
wandb.config.batch_size = batch_size
wandb.config.learning_rate = lr

wandb.init(project="Resnet50 Regression")


In [10]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs.reshape(-1), labels.float())

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.detach() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase])

            print('{} loss: {:.6f}'.format(phase, epoch_loss.item()))
            
            if phase == 'train':
                wandb.log({'Train loss':  epoch_loss.item()})
            else:
                wandb.log({'Test loss':  epoch_loss.item()})
            
            
            
    return model

In [11]:
criterion = torch.nn.HuberLoss()
optimizer = optim.Adam(model.fc.parameters(),lr)

In [12]:
model_trained = train_model(model, criterion, optimizer, num_epochs)

Epoch 1/100
----------


C:\Users\reese\miniconda3\envs\final-year-project\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


train loss: 0.168258
validation loss: 0.110150
Epoch 2/100
----------
train loss: 0.102443
validation loss: 0.075311
Epoch 3/100
----------
train loss: 0.081300
validation loss: 0.056237
Epoch 4/100
----------
train loss: 0.071439
validation loss: 0.044590
Epoch 5/100
----------
train loss: 0.071391
validation loss: 0.043381
Epoch 6/100
----------
train loss: 0.063287
validation loss: 0.039052
Epoch 7/100
----------
train loss: 0.062147
validation loss: 0.039194
Epoch 8/100
----------
train loss: 0.059201
validation loss: 0.037025
Epoch 9/100
----------
train loss: 0.059309
validation loss: 0.034286
Epoch 10/100
----------
train loss: 0.055202
validation loss: 0.043295
Epoch 11/100
----------
train loss: 0.055647
validation loss: 0.032727
Epoch 12/100
----------
train loss: 0.050801
validation loss: 0.029903
Epoch 13/100
----------
train loss: 0.054247
validation loss: 0.032168
Epoch 14/100
----------
train loss: 0.049525
validation loss: 0.029724
Epoch 15/100
----------
train loss: 0.

### 5. Save and load the model

In [13]:
save_path = 'models/pytorch/weights_run_34.h5'

In [14]:
torch.save(model_trained.state_dict(), save_path)

In [15]:
model = models.resnet101(pretrained=False).to(device)
model.fc = nn.Sequential(
               nn.Linear(2048, 2048),
               nn.ReLU(inplace=True),
               nn.Linear(2048, 1024),
               nn.ReLU(inplace=True),
               nn.Linear(1024, 128),
               nn.Identity(inplace=True),
               nn.Linear(128, 1)).to(device)
model.load_state_dict(torch.load(save_path))

<All keys matched successfully>

### 6. Make predictions on sample test images

In [16]:
img1,label1 = test_dataset.val[36]
img2,label2 = test_dataset.val[40]
img3,label3 = test_dataset.val[31]
img4,label4 = test_dataset.val[50]
img5,label5 = test_dataset.val[65]

validation_img_paths = [img1,img2,img3,img4,img5]
img_list = [Image.open(img_path) for img_path in validation_img_paths]

validation_batch = torch.stack([data_transforms['validation'](img).to(device) for img in img_list])

print(img1)
print(img2)
print(img3)
print(img4)
print(img5)

print(label1)
print(label2)
print(label3)
print(label4)
print(label5)

D:\Flow Videos\dataset\frame_612.jpg
D:\Flow Videos\dataset\frame_629.jpg
D:\Flow Videos\dataset\frame_1234.jpg
D:\Flow Videos\dataset\frame_1260.jpg
D:\Flow Videos\dataset\frame_494.jpg
0.4013
0.281
0.3027
0.4525
0.0434


In [17]:
pred_logits_tensor = model(validation_batch)

pred_logits_tensor

tensor([[0.3513],
        [0.2627],
        [0.3440],
        [0.4399],
        [0.1101]], device='cuda:0', grad_fn=<AddmmBackward>)

In [18]:
# fig, axs = plt.subplots(1, len(img_list), figsize=(20, 5))
# for i, img in enumerate(img_list):
#     ax = axs[i]
#     ax.axis('off')
#     ax.set_title("{:.0f}% bu, {:.0f}% sl, {:.0f}% ch,{:.0f}% an".format(
#     100*pred_probs[i,0], 100*pred_probs[i,1],100*pred_probs[i,2],100*pred_probs[i,3]))
#     ax.imshow(img)